In [2]:
# !git clone https://github.com/snorkel-team/snorkel-tutorials.git

Cloning into 'snorkeltutorials'...
remote: Repository not found.
fatal: repository 'https://github.com/snorkel-team/snorkeltutorials.git/' not found


In [8]:
# %cd D:\Jupyter Notebooks\SA-TextBlob-Flask

D:\Jupyter Notebooks\SA-TextBlob-Flask


In [1]:
#Import modules
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
#Check data
df = pd.read_csv("Reddit_Data.csv")
df.head()

,clean_comment,category
0,family mormon have never tried explain them t...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1


In [3]:
#Drop nan values and duplicates 
df = df.dropna()
df = df.drop_duplicates()

In [5]:
#Split into train and test
train_comments = df['clean_comment']
test_labels = df['category']  

In [6]:
#Let's generate Weak Labels with the help of Labelling Functions
from snorkel.preprocess import preprocessor
from textblob import TextBlob

POSITIVE = 1
NEUTRAL = 0
NEGATIVE = -1

In [8]:
from snorkel.labeling import labeling_function
from snorkel.labeling import PandasLFApplier
from snorkel.labeling import LFAnalysis
from snorkel.analysis import get_label_buckets
from snorkel.preprocess import preprocessor

In [44]:
# Writing an LF to gauge sentiment using TextBlob (we can use other packages with snorkel)
from textblob import TextBlob

@preprocessor(memoize=True)
def textblob_sentiment(x):
    scores = TextBlob(x['clean_comment'])
    x.polarity = scores.sentiment.polarity #Polarity == sentiment, +1 is positive, -1 is negative, 0 is a neutral sentiment.
    return x

@labeling_function(pre=[textblob_sentiment]) #If comment is positive (>0.9), then its  HAM
def textblob_polarity(x):
    if x.polarity > 0:
       return POSITIVE
    elif x.polarity == 0:
        return NEUTRAL
    else:
        return NEGATIVE

In [45]:
lfs = [textblob_polarity]

In [46]:
#Convert input to dataframe
df_train_comments = train_comments.to_frame(name='clean_comment')

In [47]:
# Apply the LFS in the train dataset (without labels = y_train)
applier = PandasLFApplier(lfs=lfs)
auto_labelled = applier.apply(df=df_train_comments)

100%|██████████████████████████████████████████████████████████████████████████| 36799/36799 [00:12<00:00, 2951.42it/s]


In [48]:
#Specially important if there are more than 1 LF
LFAnalysis(L=auto_labelled, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
textblob_polarity,0,"[0, 1]",0.775809,0.0,0.0


In [49]:
df['category'].value_counts()

category
 1    15771
 0    12778
-1     8250
Name: count, dtype: int64

In [50]:
np.unique(auto_labelled, return_counts=True)

(array([-1,  0,  1]), array([ 8250, 12778, 15771], dtype=int64))

In [53]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_labels, auto_labelled)
#the LF with polarity helped us to auto-label the whole dataset with 100% accuracy.

array([[ 8250,     0,     0],
       [    0, 12778,     0],
       [    0,     0, 15771]], dtype=int64)

In [54]:
#Lets now make 1 dataframe with the input feature (comments) & the auto-labelled target column
df = df.drop('category', axis=1)
df['sentiment'] = auto_labelled

In [55]:
df.head()

,clean_comment,sentiment
0,family mormon have never tried explain them t...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1


In [56]:
X = df['clean_comment']
y = df['sentiment']

In [57]:
#Build simple SVM model
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.calibration import CalibratedClassifierCV

vectorizer = CountVectorizer(analyzer='word')
comments = vectorizer.fit_transform(X)

lsvm = SGDClassifier(alpha=0.001, random_state=5, tol=None)
lsvm_calibrated = CalibratedClassifierCV(lsvm)

In [58]:
lsvm_calibrated.fit(comments, y)

CalibratedClassifierCV(estimator=SGDClassifier(alpha=0.001, random_state=5,
                                               tol=None))

In [59]:
import pickle
# save the model
with open("SVMModel", 'wb') as fout:
    pickle.dump((lsvm_calibrated, vectorizer), fout)

In [60]:
#load model
with open('SVMModel', 'rb') as f:
    svm_model, vectorizer = pickle.load(f)

In [61]:
messageCV = vectorizer.transform(["I LOVE HAPPY DOGS THAT KISS ME"])
pred = svm_model.predict_proba(messageCV)

In [62]:
#Order of pred results = -1, 0, 1
pred

array([[0.04702762, 0.00470012, 0.94827225]])